In [4]:
import sys, cv2
import numpy as np
import tracktor_revised as tr
import matplotlib.pyplot as plt
import pickle
import os
import collections

In [ ]:
# Open video file.
video_file        = '../Data/SF_n2_t2_3113.mp4'
cap               = cv2.VideoCapture(video_file)
n_frames          = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
fps               = cap.get(cv2.CAP_PROP_FPS)
print(f'{n_frames} frames, {fps} frames per second.')

# Contour detection parameters.
block_size,offset = 15,13
min_area,max_area = 20,1000
n_pix             = 5

# Creating dataset from file

## Manually creates the fish data set from the video file. This cell is currently disabled. 

In [ ]:
# List of frames to analyze.
#frameID_list = range(5000,50000,50)
frameID_list = range(5000,n_frames,1)

# Data structure to hold only the pixels belonging to a fish and their brightness.
fish = []

#Uncomment line to make this cell run, commented to force notebook to use the .pkl file for speed. (This cell takes over an hour.)
#for i in frameID_list:
    
    # Load the frame.
    cap.set(cv2.CAP_PROP_POS_FRAMES, i-1)
    ret,frame = cap.read()
    
    # Detect the contours.
    contours  = tr.contour_detect(frame,min_area,max_area,block_size,offset,n_pix)
    
    # Make a mask with value j inside fish number j and -1 elsewhere. 
    mask      = 0*cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY) - 1
    for j in range(len(contours)):
        cv2.drawContours(mask, contours, j, j, -1)
    
    # Make and save a list of each fish's pixels.
    pixels=[]
    subFish=[]
    for j in range(len(contours)):
        pixels=(np.nonzero(mask==j))
        n =len(pixels[0]) # Number of pixels in J
        subSubFish=[]
        for k in range(n):
            row=pixels[0][k]
            col=pixels[1][k]
            rgb=frame[row,col]
            grey=round(.3*rgb[0]+.59*rgb[1]+.11*rgb[2],2)
            subSubFish.append([row,col,grey]) #Pixel
        subFish.append(np.array(subSubFish)) #Fish
    fish.append(subFish) #Frame

with open('fish.pkl','wb') as fh:
     pickle.dump(fish,fh)

## Reads fish.pkl to create fish.

In [5]:
with open('fish.pkl','rb') as fh:
     fish=pickle.load(fh)

# Fish Overlap Check

## Checks fish for sections where the program is reading the fish as one object due to proximity, and returns a list of where the fish are not overlapping.

In [6]:
i2=0
nonOverlappingRange=[]
while i2<len(fish):
    i1=i2
    while i1<len(fish) and len(fish[i1])!=2:
        i1+=1
    i2=i1
    while i2 < len(fish) and len(fish[i2])==2:
        #find the first overlapping index
        i2+=1
    nonOverlappingRange.append([i1,i2])
print(nonOverlappingRange)

[[0, 137], [138, 204], [207, 247], [248, 371], [379, 624], [625, 916], [918, 1164], [1168, 1203], [1205, 1345], [1351, 1380], [1382, 1804], [1815, 1825], [1826, 2707], [2710, 2785], [2792, 3048], [3061, 3229], [3230, 3233], [3236, 3290], [3295, 3595], [3597, 3598], [3602, 3658], [3661, 3733], [3738, 3770], [3774, 3781], [3788, 3896], [3898, 3918], [3920, 4062], [4068, 4155], [4157, 4389], [4390, 4551], [4552, 4589], [4591, 5152], [5155, 6433], [6434, 6436], [6437, 8500], [8501, 8536], [8549, 8945], [8948, 9123], [9141, 9508], [9539, 9571], [9697, 9947], [9952, 10030], [10040, 10095], [10112, 10208], [10216, 11231], [11235, 11646], [11648, 11653], [11654, 12363], [12364, 12427], [12430, 12567], [12574, 12995], [13006, 13076], [13081, 13913], [13914, 14436], [14439, 14536], [14543, 14580], [14581, 14583], [14584, 14681], [14692, 15300], [15304, 15466], [15470, 15600], [15603, 15826], [15829, 16667], [16671, 17110], [17117, 17463], [17465, 17657], [17660, 17768], [17783, 17947], [17955, 1

## Finds the center of each fish.

In [7]:
#finding the center of each fish
#2 on fily's paper
fishMean=[]
for i in range(len(fish)):
    if len(fish[i])==1:
        fishMean.append([[np.mean(fish[i][0].T[0]),np.mean(fish[i][0].T[1])],[np.mean(fish[i][0].T[0]),np.mean(fish[i][0].T[1])]])
    else:
        fishMean.append([[np.mean(fish[i][0].T[0]),np.mean(fish[i][0].T[1])],[np.mean(fish[i][1].T[0]),np.mean(fish[i][1].T[1])]])
fishMean=np.asarray(fishMean)
#fishMean[i.j.k] is the coordianate of k of the position of fjsh j in frame i

## Measures the distance between each fish during each nonoverlapping section to vertifiy that they are the same fish.

In [8]:
def swapStatus(pos,i):
    '''
    Detext swaps between consicutivve framses based on proximity.
    
    Input:
        pos:Postionts. Array with shape (Nframes,Nfish,Ndimensions),
        i: Frame index. Int.
    
    Output:
        Int. 0 if no swaps, 1 if swapped, 2 if overlapping.
    '''
    nFish=pos.shape[1] #Number of fish
    distanceMatrix=[np.linalg.norm(pos[i+1][0]-pos[i][0]),
                    np.linalg.norm(pos[i+1][1]-pos[i][1]),
                    np.linalg.norm(pos[i+1][0]-pos[i][1]),
                    np.linalg.norm(pos[i+1][1]-pos[i][0])]
    swapCriteron=(distanceMatrix[0]+distanceMatrix[1])-(distanceMatrix[2]+distanceMatrix[3])
    if abs(swapCriteron)<1e-10:
        return 2 #Overlapping
    elif swapCriteron>0:
        return 1 #Swapped
    elif swapCriteron<0:
        return 0 #Normal
    else:
        return -1

In [9]:
def normFunc(pos,i):
    '''
    Returns the distance between all possible combinations of 4 points to check whether the on or off axis diatance is larger.
    
    Input:
        i: Index, int.
    
    Output:
        1x4 list of distances.
    '''
    return [np.linalg.norm(pos[i+1][0]-pos[i][0]),
           np.linalg.norm(pos[i+1][1]-pos[i][1]),
           np.linalg.norm(pos[i+1][0]-pos[i][1]),
           np.linalg.norm(pos[i+1][1]-pos[i][0])]


fishQuadDistance=[]
for i in range(len(fishMean)-1):
    #print(i,j)
    fishQuadDistance.append(np.array(normFunc(fishMean,i)))
fishQuadDistance=np.array(fishQuadDistance)

## Unswapping the fish

In [10]:
pos=fishMean

#Checks for swqps and creates list of swaped points
i=0
I=[]
while i < len(pos):
    j=i+1
    while j<len(pos)-1 and swapStatus(pos,j)!=1:
        j+=1
    i=j+1
    while i<len(pos)-1 and swapStatus(pos,i)!=1:
        i+=1
    I.append([j+1,i+1])
    
# As a natural consequence of how the loop is structured, it creates a pair of points that are outside of the array, which we then need to delete.
del(I[-1])

In [11]:
posU=fishMean.copy()
for j,i in I:
    posU[j:i,:]=pos[j:i,[1,0]]

status=[]
for i in range(len(posU)-1):
    status.append(swapStatus(posU,i))

In [12]:
fishQuadDistance1=[]
for i in range(len(posU)-1):
    fishQuadDistance1.append(np.array(normFunc(posU,i)))
fishQuadDistance1=np.array(fishQuadDistance1)

### Unswapping the original fish file

In [13]:
fishU=fish
for i1,i2 in I:
    for i in range(i1,i2):
        if len(fishU[i])==2:
            fishU[i][0]=fish[i][1]
            fishU[i][1]=fish[i][0]

# Dumping resultant data

Dumps the sorted fishmeans and the nonOverlappingRange in a pickle file with [posU,nonOverlappingRange]

In [14]:
sortedFish=[posU,nonOverlappingRange,fishU]

with open('sortedFish.pik','wb') as f:
    pickle.dump(sortedFish,f)